# Asset Pricing Agent

A simple asset-pricing agent, following Gourio's notes. 

The goal is the simplest possible formulation of an agent to use for asset-pricing experiments. 

Consider an agent who has utility over payoffs today and tomorrow:

$$
u(c_{t}) + \beta \mathbb{E}_{t}\left[u(c_{t+1})\right]
$$

Assume the following:

* the agent has income process $y_t$, where the first period $y_1$ is given and $y_{t+1}$ is uncertain. 
* the agent has access to an asset with payoff $x_{t+1}$
* if the asset is a stock with dividend payoff $d_t$, then total payoff for period $t$ is dividend plus the selling price of the asset at time $t$, $p_t$. That is: $$x_t = p_t + d_t$$
* let $\xi$ be the number of claims to the asset that the agent purchases in the first period

Then the consumer problem is:

$$
\begin{aligned}
\underset{\xi}{\mathrm{max}} & \;\; u(c_{t}) + \beta \mathbb{E}_{t}\left[u(c_{t+1})\right] \\
c_t & = y_t - p_t\xi \\ 
c_{t+1} & = y_{t+1} + x_{t+1}\xi
\end{aligned}
$$

We can substitute $\xi$ and find the Euler:


$$
\begin{aligned}
\dfrac{d}{d\xi} \;\; & u(y_t - p_t\xi) + \beta \mathbb{E}_{t}\left[u(y_{t+1} + x_{t+1}\xi)\right] = 0 \\
\leftrightarrow \;\; & -p_t u(c_t)' + \beta \mathbb{E}_{t}\left[u(c_{t+1})x_{t+1}\right] = 0 \\
\rightarrow \;\; & p_t u(c_t)' = \beta \mathbb{E}_{t}\left[u(c_{t+1})x_{t+1}\right]
\end{aligned}
$$


Assuming no corner solutions, this gives us our asset pricing equation in traditional form as follows: 

$$
\begin{aligned}
p_t & = \mathbb{E}_{t}\left[\beta \frac{u(c_{t+1})}{u(c_t)'}x_{t+1}\right]
\end{aligned}
$$

Let's think about the key items required for this expression:

* preference parameters: $\beta$, risk aversion $\rho$, more params if needed 
* beliefs about distibutions of:
    * dividend $d_{t+1}$
    * income $y_{t+1}$
    * price $p_{t+1}$

These requirements inform the construction of our agent. Eventually for convinience we'll create a general "belief" object, so we don't need to replicate the above items. For a first pass, however, keep it simple enough to ensure that the desired behavior is occuring. 



## Further Simplifications

Further simplifications to get a minimialistic initial model:

1. $p_{t+1}=0$ for all states. Extension: $\infty$-lived agents.
2. $y_{t+1}=0$ for all states. Extension: $\infty$-lived agents.
3. Only stochastic elements are **dividend** and **initial endowment**.
    * Let dividend take on two values, "high" and "low;" perhaps 3-point if needed. 
    * Intial endowment continuous draw. Shouldn't matter for what I think will emerge. 
4. Trade occurrs first with 1 representative agent, using two utilities:
    * risk-neutral (linear)
    * risk-averse (CRRA, $rho=2$)
5. *Then* trade occurs between agents with heterogneous beliefs.
    * Point here is to first ensure that zero-finding price discovery works.


This new most basic versio implies the following Euler:

$$
\begin{aligned}
u(c_t)'p_t & = \mathbb{E}_{t}\left[\beta u(c_{t+1})d_{t+1}\right] \\
\leftrightarrow \;\; u(y_t - p_t\xi)'p_t & = \mathbb{E}_{t}\left[\beta u(y_{t+1} + d_{t+1}\xi)d_{t+1}\right] \\
\end{aligned}
$$

Or, if ${u'}^{(-1)}$ exists, we can make the :HS and RHS equations quasi-linear in $\xi$:

$$
\begin{aligned}
y_t - p_t\xi & = {u'}^{(-1)} \left( \frac{\mathbb{E}_{t}\left[\beta u(y_{t+1} + d_{t+1}\xi)d_{t+1}\right]}{p_t} \right) \\
\end{aligned}
$$




Now on to the agent design.

In [221]:
import numpy as np
import pylab as plt
from copy import deepcopy
from scipy.optimize import brentq, newton, fixed_point

In [230]:
class AssetPricingAgent(object):
    
    def __init__(self, beta, rho, payoff_dividend, p_dividend, initial_cash_endowment=None, 
                 initial_asset_endowment=None, 
                 xi_numerical_lower_bound=-1000,
                 xi_numerical_upper_bound=1000,
                 xi_testgrid_size=2000,
                 use_semilinear=False):
        '''
        Initialize an asset-pricing agent.
        
        Parameters
        ----------
        beta : float
            Time discounting factor
        rho : float
            Risk aversion paramter
        payoff_dividend : array_like(float, ndim=2 or 1)
            Payoff values for different states of the dividend
        p_dividend : array_like(float, ndim=2 or 1)
            Probabilities associated with payoffs for states in payoff_dividend vector
        initial_endowment : float
            Initial endowment

        Returns
        -------
        Nothing
        '''
        
        # Set agent paramters:
        self.beta, self.rho = beta, rho
        self.payoff_dividend = deepcopy(payoff_dividend)
        self.D = self.payoff_dividend
        self.p_dividend = deepcopy(p_dividend)
        self.D_probs = self.p_dividend
        self.initial_cash_endowment = initial_cash_endowment
        #self.initial_asset_endowment = initial_asset_endowment
        
        self.use_semilinear = use_semilinear
        
        # Quick check:
        assert np.isclose(np.sum(p_dividend), 1.0), "Problem: p_dividend does not sum to 1.0: np.sum(p_dividend) = " + str(np.sum(p_dividend))

    
        # Set CRRA utility:
        if rho == 1.0:
            self.u = np.log
            self.uprime = lambda c: 1.0/c
            self.uprimeprime = lambda c: -c**(-2)
            self.uprime_inverse = lambda z: 1.0/z
        elif rho == 0.0:
            # Then constant utility - risk neutral
            self.u = lambda c: c
            self.uprime = lambda c: 1.0
            self.uprimeprime = lambda c: 0.0
            self.uprime_inverse = None # Not defined for constant function
        else:
            self.u = lambda c, oneminusrho=(1.0-self.rho): c**oneminusrho / oneminusrho
            self.uprime = lambda c, rho=self.rho: c**-rho
            self.uprimeprime = lambda c, rho=self.rho: -rho*c**(-1.0-rho)
            self.uprime_inverse = lambda z, rhoinv=(1.0/self.rho): z**-rhoinv

        # Define two sides of Euler, to solve demand:
        self.EulerLHS, self.EulerRHS,  self.EulerLHS_prime, self.EulerRHS_prime = self.set_Euler(self.initial_cash_endowment)

        # Define an initial guess at xi:
        self.xi0_guess = (self.initial_cash_endowment - np.dot(payoff_dividend, p_dividend))/2.0
            # Comes from approximation of demand in risk-neutral case
        print "self.xi0_guess =",self.xi0_guess
        
        
        # Set up machinery to figure out where xi demand falls:
        self.xi_numerical_lower_bound = xi_numerical_lower_bound
        self.xi_numerical_upper_bound = xi_numerical_upper_bound
        self.xi_testgrid_size = xi_testgrid_size
        assert self.xi_numerical_lower_bound < 0, "self.xi_numerical_lower_bound not < 0: self.xi_numerical_lower_bound = "+str(self.xi_numerical_lower_bound)
        assert self.xi_numerical_upper_bound > 0, "self.xi_numerical_upper_bound not > 0: self.xi_numerical_upper_bound = "+str(self.xi_numerical_upper_bound)

        # Now set up two grids:
        self.xi_testgrid_less_than_0 = np.linspace(self.xi_numerical_lower_bound, 0.0, self.xi_testgrid_size)
        self.xi_testgrid_greater_than_0 = np.linspace(0.0, self.xi_numerical_upper_bound, self.xi_testgrid_size)
        
        

    def set_endowment(self, y0):
        self.initial_cash_endowment = y0
        self.EulerLHS, self.EulerRHS,  self.EulerLHS_prime, self.EulerRHS_prime = self.set_Euler(y0)


    def set_Euler(self, y0):
        
        def EulerLHS(xi, p, y0=y0):
            return np.log(self.uprime(y0 - p*xi)*p)

        def EulerRHS(xi, p, y0=y0, beta=self.beta, D=self.payoff_dividend, Pr=self.p_dividend): 

            Uprime = self.uprime(xi*D)*D

            return np.log(beta*np.dot(Uprime, Pr))


        EulerLHS_prime = None
        EulerRHS_prime = None
        
        return EulerLHS, EulerRHS, EulerLHS_prime, EulerRHS_prime
        
        
        
    def set_Euler(self, y0):
        # Note: change these to get simpler/better zero-finding.
        #EulerLHS = lambda xi, p, y0=y0: self.uprime(y0-p*xi)*p
        #EulerRHS = lambda xi, p, y0=y0, beta=self.beta, D=self.payoff_dividend, Pr=self.p_dividend: beta*np.dot(self.uprime(xi*D)*D, Pr)
        
        # This version gives us nearly-linear functions, which are much easier to zero-find on:
        #EulerLHS = lambda xi, p, y0=y0: y0-p*xi
        #EulerRHS = lambda xi, p, y0=y0, beta=self.beta, D=self.payoff_dividend, Pr=self.p_dividend: self.uprime_inverse(beta * np.dot( self.uprime(xi * D)*D, Pr) / p )

        if not self.use_semilinear:
            def EulerLHS(xi, p, y0=y0):
                
                return self.uprime(y0 - p*xi)*p
            #    return p


            def EulerRHS(xi, p, y0=y0, beta=self.beta, D=self.payoff_dividend, Pr=self.p_dividend): 

                #print "xi = ",xi
                #print "D = ",D
                #print "Pr = ",Pr
                
                Uprime = self.uprime(xi*D)*D
                
                return beta*np.dot(Uprime, Pr)
            #    return beta*np.dot(Uprime, Pr) / self.uprime(y0-p*xi)

            
            def EulerLHS_prime(xi, p, y0=y0):
                
                return self.uprimeprime(y0 - p*xi)*p*p


            def EulerRHS_prime(xi, p, y0=y0, beta=self.beta, D=self.payoff_dividend, Pr=self.p_dividend): 

                Uprimeprime = self.uprimeprime(xi*D)*D*D
                
                return beta*np.dot(Uprimeprime, Pr)
                
        else:
        
            def EulerLHS(xi, p, y0=y0):

                return p


            def EulerRHS(xi, p, y0=y0, beta=self.beta, D=self.payoff_dividend, Pr=self.p_dividend): 

                Uprime = self.uprime(xi*D)*D

                UprimeInversed = self.uprime_inverse(beta*np.dot(Uprime, Pr))

                return UprimeInversed / (y0-p*xi)
            
            EulerLHS_prime = None
            EulerRHS_prime = None
        
        return EulerLHS, EulerRHS, EulerLHS_prime, EulerRHS_prime


    
    def demand(self, p):
        '''
        Given a price p, use agent first-order conditions and expectations to determine demand. 

        Parameters
        ----------
        p : float
            possible price 

        Returns
        -------
        xi : float
            Demand at price p 
        '''

        '''
        # Define the function for which we want to find fixed point:
        
            xi =  (self.y0 - self.uprime_inverse(self.beta / p * np.dot(self.uprime(self.D*xi)*self.D, self.D_probs) ) ) / p
        '''
        
        
        xi_fixed_point = lambda xi, p=p: (self.initial_cash_endowment - self.uprime_inverse(self.beta / p * np.dot(self.uprime(self.D*xi)*self.D, self.D_probs) ) ) / p
        
        xi_choice = fixed_point(func=xi_fixed_point, x0=self.xi0_guess+1e-4, args=(), xtol=1e-08, maxiter=500)


        
    def demand2(self, p):
        '''
        Given a price p, use agent first-order conditions and expectations to determine demand. 

        Parameters
        ----------
        p : float
            possible price 

        Returns
        -------
        xi : float
            Demand at price p 
        '''

        # Define the function to zero-find on:
        obj = lambda xi, q=p: np.log(self.EulerLHS(xi, q)) - np.log(self.EulerRHS(xi, q))
        
        x0, root_results = brentq(f=obj, a=0.0, b=self.xi_numerical_upper_bound , xtol=1e-16, rtol=4.4408920985006262e-16, args=(p), full_output=True, disp=True)

        if not root_results.converged:
            print "WARNING: root_results.converged not True!:  root_results.converged = ", root_results.converged
            print "root_results:", root_results

        return x0
        
        
        
    def demand3(self, p):
        '''
        Given a price p, use agent first-order conditions and expectations to determine demand. 
        
        Parameters
        ----------
        p : float
            possible price 

        Returns
        -------
        xi : float
            Demand at price p 
        '''

        # Define the function to zero-find on:
        obj = lambda xi, q=p: np.log(self.EulerLHS(xi, q)) - np.log(self.EulerRHS(xi, q))
        
        
        # Determine which side of zero it falls on:
        '''
        # Note: can't do the follwoing directly because haven't set up the vectorization of EulerLHS/EulerRHS above, so can't just drop over a grid.
        obj_less_than_0 = obj(self.xi_testgrid_less_than_0)
        max_obj_less_than_0 = np.max(obj(obj_less_than_0))
        min_obj_less_than_0 = np.min(obj(obj_less_than_0))

        obj_greater_than_0 = obj(self.xi_testgrid_greater_than_0)
        max_obj_greater_than_0 = np.max(obj(obj_greater_than_0))
        min_obj_greater_than_0 = np.min(obj(obj_greater_than_0))

        
        xi_crosses_over_less_than_0 = ((max_obj_less_than_0 > 0 and min_obj_less_than_0 < 0) or 
                                       (max_obj_less_than_0 < 0 and min_obj_less_than_0 > 0))

        xi_crosses_over_greater_than_0 = ((max_obj_greater_than_0 > 0 and min_obj_greater_than_0 < 0) or 
                                          (max_obj_greater_than_0 < 0 and min_obj_greater_than_0 > 0))
                                       
        assert xi_crosses_over_less_than_0 != xi_crosses_over_greater_than_0, "xi_crosses_over_less_than_0 == xi_crosses_over_greater_than_0:\n\t xi_crosses_over_less_than_0 = "+str(xi_crosses_over_less_than_0) +"\n\t xi_crosses_over_greater_than_0 = "+str(xi_crosses_over_greater_than_0)
        '''
        
        
        obj_lower_bound_less_than_0 = obj(self.xi_numerical_lower_bound)
        obj_epsilon_less_than_0 = obj(-1e-5)

        obj_lower_bound_greater_than_0 = obj(self.xi_numerical_upper_bound)
        obj_epsilon_greater_than_0 = obj(1e-5)

        
        xi_crosses_over_less_than_0 = ((obj_lower_bound_less_than_0 > 0 and obj_epsilon_less_than_0 < 0) or 
                                       (obj_lower_bound_less_than_0 < 0 and obj_epsilon_less_than_0 > 0))

        xi_crosses_over_greater_than_0 = ((obj_lower_bound_greater_than_0 > 0 and obj_epsilon_greater_than_0 < 0) or 
                                          (obj_lower_bound_greater_than_0 < 0 and obj_epsilon_greater_than_0 > 0))
                                       
        #assert xi_crosses_over_less_than_0 != xi_crosses_over_greater_than_0, "xi_crosses_over_less_than_0 == xi_crosses_over_greater_than_0:\n\t xi_crosses_over_less_than_0 = "+str(xi_crosses_over_less_than_0) +"\n\t xi_crosses_over_greater_than_0 = "+str(xi_crosses_over_greater_than_0)
        # NOTE: NOT BEST, fix!
        
        if xi_crosses_over_less_than_0:
            print "GOT HERE: xi_crosses_over_less_than_0 ========================================"
            # Now zero-find:        
            #x0, root_results = brentq(f=obj, a=-self.initial_asset_endowment, b=1000, xtol=1e-16, rtol=4.4408920985006262e-16, args=(p), full_output=True, disp=True)
            x0, root_results = brentq(f=obj, a=self.xi_numerical_lower_bound, b=0.0, xtol=1e-16, rtol=4.4408920985006262e-16, args=(p), full_output=True, disp=True)
        elif xi_crosses_over_greater_than_0:
        #else:
            print "GOT HERE: xi_crosses_over_greater_than_0 ++++++++++++++++++++++++++++++++++++++++"
            # Now zero-find:        
            #x0, root_results = brentq(f=obj, a=-self.initial_asset_endowment, b=1000, xtol=1e-16, rtol=4.4408920985006262e-16, args=(p), full_output=True, disp=True)
            x0, root_results = brentq(f=obj, a=0.0, b=self.xi_numerical_upper_bound , xtol=1e-16, rtol=4.4408920985006262e-16, args=(p), full_output=True, disp=True)
        else:
            x0 = 0.0
            print "warning: no demand. Come back and fix."
            root_results = lambda x:x
            root_results.converged = True
            
        if not root_results.converged:
            print "WARNING: root_results.converged not True!:  root_results.converged = ", root_results.converged
            print "root_results:", root_results
            
        # NOTE: Problem is that there is a *sharp discontinuity* at xi=0.0
        # SO essentially need to do two different optimizations.
        # 1. First determine which side crossing happens on -- xi >= 0 or xi <= 0
        # 2. Then determine where crossing is. 
        #
        # "Cheap"/simple way to figure out where crossing occurs is use a fine-ish grid and test
        # Will need to think about how to do this better, later!
        # TODO: Solve "how to do this better."
            
            
        
        '''
        
        obj = lambda xi, q=p: self.EulerRHS(xi, q)/self.EulerLHS(xi, q) - 1

        obj_prime = lambda xi, q=p: self.EulerRHS_prime(xi, q) - self.EulerLHS_prime(xi, q)

        
        x0 = newton(obj, x0=self.xi0_guess, tol=1.48e-08, maxiter=50, fprime2=None)
        #x0 = newton(obj, x0=self.xi0_guess, fprime=obj_prime, args=(), tol=1.48e-08, maxiter=50, fprime2=None)
        '''
        
        return x0


In [231]:
# Now let's define a market which will find the price given a set of agents:
class AssetMarket(object):
    
    def __init__(self, agents, aggregate_asset_endowment=1.0):
        '''
        Market simply consists of collection of agents.
        '''
        self.agents = agents
        self.aggregate_asset_endowment = aggregate_asset_endowment
        
        # To find reasonable "first guess" price, find the risk-neutral
        # asset price for first-agent's preferences:
        agent0 = self.agents[0]
        self.p0_guess = agent0.beta * np.dot(agent0.payoff_dividend, agent0.p_dividend)
        
        
    def get_excess_demand(self, p):
        '''
        Iterate over all agents and ask for demand given price.
        '''
        
        total_demand = 0.0
        for an_agent in self.agents:
            total_demand += an_agent.demand(p)
        
        return(total_demand-self.aggregate_asset_endowment)
        

    def find_market_clearing_price(self, p0=None):
        '''
        Given an initial price guess, zero-find on total asset demand.
        
        p0 is initial price. 
        '''
        
        # Set intial price guess
        if p0 is None:
            p0 = self.p0_guess
            # Note: currently not using first guess....
        
        # Zero-find to determine price:
        p, root_results = brentq(f=self.get_excess_demand, a=0.001, b=1000, full_output=True, disp=True)
        
        if not root_results.converged:
            print "WARNING: root_results.converged not True!:  root_results.converged = ", root_results.converged
            print "root_results:", root_results
        
        return p


Now set up a couple agents and see what the results are:

In [233]:
# Set up an agent and a dividend:
beta=0.99
payoff_dividend = np.array([0.7, 1.23, 1.9])
p_dividend = np.array([0.2,0.6,0.2])
initial_cash_endowment = 1.0
rho=1.0
use_semilinear=False
total_endowment = 1.0

# Define an agent:
an_agent = AssetPricingAgent(beta=beta, 
                             rho=rho, 
                             payoff_dividend=payoff_dividend,
                             p_dividend=p_dividend, 
                             initial_cash_endowment=initial_cash_endowment,
                             use_semilinear=use_semilinear)

# Now explore the agent a little
# For a few prices, plot the LHS and RHS Eulers for a set xi values
prices = np.linspace(1e-5,10,20)


# Let's find the demand function at various prices:
demands = []
for p in prices:
    demands.append(an_agent.demand2(p))

plt.plot(demands, prices)
plt.xlabel("xi demanded")
plt.ylabel("Prices")
plt.show()


print "prices", prices
print "demands", demands

'''
xi_values = np.linspace(-50,50, 400)
prices = [0.01, 0.5, 1.2]
LHS_functions = []
RHS_functions = [] 
for q in prices:
    LHS = []
    RHS = []
    LHS_minus_RHS = []
    for xi in xi_values:
        LHS.append(an_agent.EulerLHS(xi, q))
        RHS.append(an_agent.EulerRHS(xi, q))
        LHS_minus_RHS.append( an_agent.EulerLHS(xi, q) - an_agent.EulerRHS(xi, q) )
    
    LHS_functions.append(LHS)
    RHS_functions.append(RHS)

    plt.plot(xi_values, LHS, 'b-', label="LHS")
    plt.plot(xi_values, RHS, 'r-', label="RHS")
    
    plt.legend()
    plt.show()
    plt.close()
    

    plt.plot(xi_values, LHS_minus_RHS, 'b-', label="LHS-RHS")
    plt.plot(xi_values, [0.0]*len(LHS_minus_RHS), 'k--', label="Zero")
    plt.legend()
    plt.show()
    plt.close()
'''

self.xi0_guess = -0.129


/home/npalmer/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:48: RuntimeWarning: divide by zero encountered in divide


ValueError: f(a) and f(b) must have different signs

In [ ]:
# Define a market:
a_market = AssetMarket([an_agent], total_endowment)


# Let's get market demand for a set of differnt values:
prices = []
demands = []
for p in np.linspace(0.2, 10, 100):
    prices.append(p)
    demands.append(a_market.get_excess_demand(p))

plt.plot(prices, demands)
plt.plot(prices, np.zeros_like(prices))
plt.xlabel("Prices")
plt.xlabel("Excess Asset Demand")
plt.show()

# Find price:
price = a_market.find_market_clearing_price()

print "Market clearing price:", price


In [180]:
X = [-111111.23, 3.0]
np.sum(np.sign(X))
%timeit np.sum(np.sign(X)) == 0

The slowest run took 8.75 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 8.58 µs per loop


In [181]:
np.sum(np.sign(X)) == 0

True

In [182]:
x0=X[0]
x1=X[1]
%timeit (x0 > 0 and x1 < 0) or (x0 < 0 and x1 > 0) 

The slowest run took 10.27 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 302 ns per loop


In [74]:
# Define a market:
a_market = AssetMarket([an_agent])


# Let's get market demand for a set of differnt values:
prices = []
demands = []
for p in np.linspace(0.1, 10, 200):
    prices.append(p)
    demands.append(a_market.get_excess_demand(p))

plt.plot(prices, demands)
plt.show()
# Find price:
#price = a_market.find_market_clearing_price()

#print "Market clearing price:", price


In [64]:
Rho = 2.0
uprime = lambda c, rho=Rho: 1.0/(c**rho)
uprime_inverse = lambda z, rhoinv=(1.0/Rho): 1.0/(z**rhoinv)

val = 4.232
print "uprime("+str(val)+") = ", uprime(val)
print "uprime_inverse(uprime("+str(val)+")) = ", uprime_inverse(uprime(val))





uprime(4.232) =  0.0558352778899
uprime_inverse(uprime(4.232)) =  4.232


Here is the log utility version of extracting the demand function:

$$
\begin{aligned}
u'(c_t)p \; &= \beta \mathbb{E}\left[u'(c_{t+1})x_{t+1} \right] \\
c_t^{-1}p \; &= \beta \mathbb{E}\left[c_{t+1}^{-1}x_{t+1} \right] \\
(y-p\xi)^{-1}p \; &= \beta \mathbb{E}\left[(d\xi)^{-1}d \right] \\
(y-p\xi)^{-1} \; &= \frac{\xi^{-1}\beta}{p} \\
\frac{\xi}{y-p\xi} \; &= \frac{\beta}{p} \\
\frac{y-p\xi}{\xi} \; &= \frac{p}{\beta} \\
\frac{y}{\xi}-p \; &= \frac{p}{\beta} \\
\frac{y}{\xi}\; &= \frac{p}{\beta} + p \\
\frac{y}{\xi}\; &= p\frac{1+\beta}{\beta} \\
\frac{y}{\xi}\; &= p\frac{1+\beta}{\beta} \\
\xi \; &= \frac{y}{p}\frac{\beta}{1+\beta} \\
\end{aligned}
$$

Now the full CRRA specification:

$$
\begin{aligned}
u'(c_t)p \; &= \beta \mathbb{E}\left[u'(c_{t+1})x_{t+1} \right] \\
c_t^{-\rho}p \; &= \beta \mathbb{E}\left[c_{t+1}^{-\rho}x_{t+1} \right] \\
(y-p\xi)^{-\rho}p \; &= \beta \mathbb{E}\left[(d\xi)^{-\rho}d \right] \\
(y-p\xi)^{-\rho} \; &= \xi^{-\rho}\frac{\beta \mathbb{E}\left[d^{(1-\rho)} \right]}{p} \\
\left(\frac{y-p\xi}{\xi} \right)^{-\rho} \; &= \frac{\beta \mathbb{E}\left[d^{(1-\rho)} \right]}{p} \\
\frac{y-p\xi}{\xi} \; &= \left(\frac{\beta \mathbb{E}\left[d^{(1-\rho)} \right]}{p} \right)^{-\frac{1}{\rho}}\\
\frac{y}{\xi} \; &= \left(\frac{\beta \mathbb{E}\left[d^{(1-\rho)} \right]}{p} \right)^{-\frac{1}{\rho}}+p\\
\xi \; &= \frac{y}{\left(\frac{\beta \mathbb{E}\left[d^{(1-\rho)} \right]}{p} \right)^{-\frac{1}{\rho}}+p}\\
\xi \; &= y\left(\left(\frac{\beta \mathbb{E}\left[d^{(1-\rho)} \right]}{p} \right)^{-\frac{1}{\rho}}+p \right) ^{-1}\\
\xi \; &= y\left(\left(\frac{\beta \mathbb{E}\left[d^{(1-\rho)} \right]}{p} +p^{-\rho} \right)^{-\frac{1}{\rho}} \right) ^{-1}\\
\xi \; &= y\left(\frac{\beta \mathbb{E}\left[d^{(1-\rho)} \right]}{p} +p^{-\rho} \right)^{\frac{1}{\rho}}\\
\end{aligned}
$$

In [240]:
# Now enter amounts:
# Set up an agent and a dividend:
beta=0.99
D = np.array([0.7, 1.23, 1.9])
D_prob = np.array([0.2,0.6,0.2])
y = 1.0
rho=2.0
use_semilinear=False
total_endowment = 1.0

def demand(p,y=y,beta=beta,D=D,D_prob=D_prob,rho=rho):
    return y * (beta * np.dot(D**(1.0 - rho), D_prob) / p + p**(-rho) ) ** (1.0/rho)

def demand_log(p,y=y,beta=beta,D=D,D_prob=D_prob,rho=rho):
    return y / p * beta/(1+beta) 


prices = np.linspace(0.1,10,100)
demands = []
logdemands = []

for p in prices:
    demands.append(demand(p))
    logdemands.append(demand_log(p))

plt.plot(demands,prices)
plt.plot(logdemands,prices)
plt.xlabel("Quantity Demanded")
plt.ylabel("Prices")
plt.show()
plt.close()



# Remember, deflate the notebook